# Final project

## Project title

Research Question: Which UvA courses do I meet the entry requirements for? XXX






## Process Book

1. Don’t forget the process book. Maintain it every day you’ve worked on the project. This is the main way for 
   us to see what you’ve learned and is an important part of the evaluation.
2. Don’t use it for everything. Don’t spend too much time on your process book! Ten minutes 
   every day you’ve worked on the project on this, should be more than enough.
3. Keep it simple and short. Use simple language, short sentences, and be as straightforward as possible when 
   documenting your learning process.
4. Create a simple template. In order to organize your process book, it is nice to create and use a template:


### Date of entry: 25. Jan
#### What I’ve worked on: 
Git
- Lecture git
- Tried out git by myself
- Created git repository

Project choice
- Learn to use tableau and uber data as I am applying to intern at uber and tableau is a desired skill

Tableau
- Downloaded tableau 
- See what I can use tableau for to decide which project I want to do specifically
- Learn tableau basics
- Experiment with DP datasets (e.g., weather data) in tableau
- Experiment with uber selection dataset --> too easy (?)
    
#### What problems I encountered:
- Cannot use dataset I first thought about using

#### What I learned:
- How to use git
- Tableau basics https://www.youtube.com/watch?v=jEgVto5QME8
- I want to gather my own uber data e.g., through web-scraping, or API

#### Which resources did I use:
- Course material, YouTube, tableau, tableau user guide, uber data XXX




### Date of entry: 26. Jan
#### What I’ve worked on: 
Tableau
- Learned more functions of tableau and experimented with these  

Project
- Created jupyter notebook
- Started process book

Brainstorm
- Potentially make website with tableau dashboard
- Which data can I use? 

Find data
- Look for uber APIs

#### What problems I encountered: 
- Uber APIs are not open, I can not use them 

#### What I learned: 
Tableau
- Basic functions: https://www.youtube.com/watch?v=yejulE4b-3Y
- Use and join (csv) files: https://www.youtube.com/watch?v=5WXrJ1JvTOo
- Integrate tableau dashboard in website: https://www.youtube.com/watch?v=wJ2CHIJalNU

#### Which resources did I use:
- Course material, YouTube, tableau, tableau user guide



### Date of entry: 27. Jan
#### What I’ve worked on: 
- Find new project ideas
- Decided for final topic/Question
- Changed strategy for scraping data: work with individual course pages instead of course overview

#### What problems I encountered: 
- Url of studiegids does not change per page of courses

#### What I learned: 

#### Which resources did I use:
- Multiple forms to try to solve the url problem, but didn't find a solution




### Date of entry: 30. Jan
#### What I’ve worked on: 
- Scraping course data: name of course, url, fiel of studies, education level, entry requirements
- Started transforming the entry requirements text into categorical entry requirement checks that students can filter for

#### What problems I encountered: 
- I cannot simply check whether a unique number-combination in the end of the link opens a specific course page by testing if the link is dead or not: if the number in the end doesn't link to a course it still opens the studigids page
- The entry requirement texts are very heterogenous and need a lot of transformation. This will become the most time-consuming part of scraping


#### What I learned: 
- New commands for text transformation

#### Which resources did I use:





### Date of entry: 31. Jan
#### What I’ve worked on: 
Refined scraping entry requirements (scraper 2):
- Added function (1) to prepare/transform entry requirements text for further functions
- Added function (2) to extract courses and with specific credit requirements
- Added function (3) to extract all courses required 
- Combined these functions in one entry requirements-scraper

#### What problems I encountered: 
- Function (3) often required adjustments as it recognised some 'course names' incorrectly
- I could integrate a feedback function

#### What I learned: 

#### Which resources did I use:
- nltk to detect stopwords




### Date of entry: 1. Feb
#### What I’ve worked on: 
Refined scraping entry requirements (scraper 2):
- Improved function (2) to extract courses and with specific credit requirements
- Experimented with combining scraper part 1 with scraper part 2 (entry requirements)

#### What problems I encountered: 
- Transform entry requirements text to categories: Special cases for scraping course that has credit requirement after the course name ('course name (XX EC)') instead of before ('XX ec in course name)

#### What I learned: 
- New commands for text processing
- New ways of slicing/working with lists

#### Which resources did I use:



### Date of entry: 2. Feb
#### What I’ve worked on: 
- Making final combined scraper
- Fixing entry requirements errors

#### What problems I encountered: 

#### What I learned: 

#### Which resources did I use:

forums
stackoverflow



### Date of entry: 11. May
#### What I’ve worked on: 
- Fixing credit requirements errors

#### What problems I encountered: 
- Error in credit requirement function when last word capitalised (index out of range was checked) e.g., ['48 ec uit een propedeuse Geesteswetenschappen'] (fixed)
- Fail of credit requirement function when no course name can be found (fixed)

#### What I learned: 
- Checking if variable exists

#### Which resources did I use:




### Date of entry: 28. May
#### What I’ve worked on: 
- Prepared filter and eligibility check for GUI

#### What problems I encountered: 

#### What I learned:
- preparing data for a GUI

#### Which resources did I use:




### Date of entry: 29. May
#### What I’ve worked on: 
- GUI 
    - worked on functionality

#### What problems I encountered: 

#### What I learned:
- new package tkinter
    - adding new windows, dropdowns, 'labels', buttons

#### Which resources did I use:
- tkinter documentation



### Date of entry: 1. June
#### What I’ve worked on: 
- GUI
    - design

#### What problems I encountered: 

#### What I learned: 
- tkinter
    - design elements, like font, padding etc

#### Which resources did I use:

## To-Do

1. Scrape / Transform data 
2. Import data into tableau
3. (Webpage with tableau dashboard)

## 1. Data

Relevant personal data

Programme: ""
Level: Bachelor's, Master's (Master's, Master), Exchange programme ('Exchange Programme Exchange Programme UvA', 'Exchange Programme Exchange Programme'), pre-Master's programme, Secondary Subject/Bijvak('?'), 'Honours MSc', 'Research Master's
Year of the studies: ""
Credits obtained this/1st year: ""

url: https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course


Relevant course information 

Course name: Course name
Entry requirements: Course > Entry requirements > Split by sentence & look for keywords (if not present, add to dict with unknown entry requirements)
Level: Course > Is part of > loop through Level list (s.a.) 
Field: Course > Is part of > remove Level > everything left 

(Coordinator name: Course > Lecturer(s) > Coordinator ('co-ordinator')
(Coordinator email: Course > Lecturer(s) > Coordinator (crawl) > Email)


In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing

from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
import numpy as np
from math import ceil
import sys
import nltk
nltk.download('punkt')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lbecker\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lbecker\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#Prepare webscraping with BeautifulSoup

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
        }
        with closing(get(url, stream=True, headers=headers)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        print('The following error occurred during HTTP GET request to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
            and content_type is not None
            and content_type.find('html') > -1)

- All courses have individual descriptions with unique urls
- First, I will find all urls that link to courses by trying out urls
- I can check whether I found all courses by comparing the length of my url list to the number or courses stated on the website
- After that, I will loop through the urls that work and scrape the data for all courses

In [3]:
#SCRAPER PART 1: name, urls, levels and fields (subset of the data)
#Delete (?)

potential_levels = ["Bachelor's", "Dual Master's", "Master's", "Exchange Programme Exchange Programme UvA", 
                    "Exchange programme Exchange Programme", "Exchange programme Exchange Programme UvA", 
                    "Exchange Programme Exchange Programme", "pre-Master's programme", "Secondary Subject", 
                    "Bijvak", "Honours MSc", "Research Master's", "Minor"] #tailored to english website

courses = {}
unique_n = 102590
    
while unique_n < 102600:
    url = f'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/{unique_n}'
    html = simple_get(url)
    dom = BeautifulSoup(html, 'html.parser')
    
    unique_n += 1
    
    if (dom.find('article', class_ = 'twelve columns')) is None: #no course under this url - go to next url
        pass
    
    else: #course exists
        name = dom.find('article', class_ = 'twelve columns').h1.text[1::]
        courses[name] = {}
        #print(name)
        courses[name]['url'] = url
        courses[name]['Academic level'] = [] #Add empty list in case there is no data
        courses[name]['Field of study'] = [] #Add empty list in case there is no data
        courses[name]['Entry requirements'] = [] #Add empty list in case there is no data

        #print(url)
        
        for info_block in dom.find_all('tr', {'class':''}):
            info_block = info_block.text


            #Education level and field of the course
            if 'Is part of' in info_block: #Find text box that contains level and field of the course
                is_part_of = info_block.replace("Is part of", "") #Remove title of the text box

                # Education level of the course e.g., Bachelor's
                levels = [] 
                for l in potential_levels:
                    if l in is_part_of:
                        levels.append(l)

                # Field(s) of the course e.g., Business
                for lev in levels: 
                    is_part_of = is_part_of.replace(lev+" ", ";") #replace education level by semicolon to split on the semicolon later
                
                fields = is_part_of[1::].split(";") #remove first semicolon to not get an empty element in the study fields list 
                                                    #and split on semicolon to get this list
        
                courses[name]['Academic level'] = levels
                courses[name]['Field of study'] = fields


print(courses)

{'Work, Power & Income:\xa0Why regulation fails to address labour market inequality (&sometimes succeeds)': {'url': 'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102591', 'Academic level': ["Bachelor's"], 'Field of study': ['Politics, Psychology, Law and Economics (PPLE)'], 'Entry requirements': []}, 'American Think Tanks and U.S. Foreign Policy ': {'url': 'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102593', 'Academic level': ['Exchange programme Exchange Programme'], 'Field of study': ['Graduate School of Humanities (MA)'], 'Entry requirements': []}, '"Losing Earth"?: Activism and Diplomacy on the Environment and Climate since 1968': {'url': 'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102595', 'Academic level': ['Exchange programme Exchange Programme'], 'Field of study': ['Graduate School of Humanities (MA)'], 'Entry requirements': []}}


In [5]:
# LOWERCASE FIRST/SECOND/THIRD YEAR AND ECS - finished

def clean_entry(entry):
    low = ['Year', 'EC', 'First', 'Second', 'Third', 'First-Year', 'First-year', 'first-Year', # List of words that need to be lowercase
           'Second-Year', 'Second-year', 'second-Year', 'Third-Year', 'Third-year', 'third-Year',
           'EC', 'ECTs', 'Credits', 'ECTS', 'ECT', 'Ect', 'Ec', 'ECs', 'Ecs'] 
    low_2 = ['EC)', 'ECTs)', 'Credits)', 'ECTS)', 'ECT)', 'Ect)', 'Ec)', 'ECs)', 'Ecs)'] 
    low_3 = ['EC),', 'ECTs),', 'Credits),', 'ECTS),', 'ECT),', 'Ect),', 'Ec),', 'ECs),', 'Ecs),']
    error_wordsl = ['training:'] #words I found that were incorrectly lowercased

    entry_c = '' #transformed entry
    #print(entry)

    
    for sentence in entry:
        words = sentence.split() #split entry into sentences and into words

        for word in words:
            if word in low:
                word_l = word.lower() #Transform first/second/third year to lowercase so it doesn't interrupt the course title detection
                entry_c += word_l+" "
            elif word in low_2:
                word_l = word.lower()
                word_l = word_l[:-1]
                entry_c += word_l+" "
            elif word in low_3:
                word_l = word.lower()
                word_l = word_l[:-2] #XXX offene Klammer; dann nach links anstatt rechts
                entry_c += word_l+" "
            elif word in error_wordsl:
                word_l = word[0].upper() + word[1:]
                entry_c += word_l+" "
            else:
                entry_c += word+ " "
        entry_c = entry_c[:-1]+"." #remove last space

    entry_c = entry_c.split('.') #turn entry form full-text into list of sentences again
    entry_c = list(filter(None, entry_c)) #remove empty elements

    return entry_c


In [6]:
#FUNCTION: COURSE TITLES 

# Extract list of course titles from the entry requirements text; based on capitalisation; sensitive to words
# that are typically not capitalised in titles (conjunctions, articles, prepositions)

def find_c_names(words): #input: list of words of one sentence from the entry requirements text
    
    # List of words not capitalised in headings
    conjunctions = ["and", "or", "but", "nor", "yet", "so", "for", "&"]
    articles = ['a', 'an', 'the']
    prepositions = ["in", "to", "of", "at", "by", "up", "for", "off", "on"]
    years = ['year', 'first-year', 'second-year', 'third-year'] 
    years_cond = ['first', 'second', 'third', '1st', '2nd', '3rd', 'academic', 'Academic'] #needs to be separate because of problem: Clinical Psychology and the first
    no_cap = conjunctions + articles + prepositions
    stop_words = set(nltk.corpus.stopwords.words("english")) #set of english stopwords
    error_wordsu = ['Students', 'Course', '“Recommended', 'Knowledge”'] #words that were incorrectly capitalised

    
    
    string = ''
    
    # Find wordgroups of nouns and conjunctions/articles/prepositions
    for i in range(len(words)):
        if words[i][0] in upper or words[i] in no_cap or words[i] in years or words[i] in years_cond:
            string += words[i] + " "
            #print(string)
        else:
            string = string[:-1]
            string += "=" 
    string = string[:-1] #remove last split sign (=)
    c_names_raw = string.split('=')
    
    c_names = []
    
    
    
    # Delete non-capitalised words that embed capitalised words - these are likely not part of the course title (e.g., 'of the', 'Clinical Psychology and the', 'of Evidence-based Clinical Practice: Research Methods/CP')
    for ele in c_names_raw:
        
        words = ele.split()
        #print(f'words:{words}')
        
        while len(words) != 0 and words[0][0] not in upper and words[0] not in years and words[0] not in years_cond: #delete no cap words in the beginning
            del words[0]

        while len(words) != 0 and words[-1][0] not in upper and words[-1] not in years: #delete no cap words in the end and 'first'/'second'/'third' if not used in context of a year
            del words[-1]

        if len(words) == 1 and words[0].lower() in no_cap or len(words) == 1 and words[0].lower() in stop_words or len(words) == 1 and words[0] in error_wordsu: #delete single no cap words and stop words that were capitalised as they opened a sentence XXXtechnically, all verbs and adjectives are probably not course titles
            del words[0]
            
        ele = ' '.join(words)
        
        if len(ele) != 0: #do not append empty course names
            c_names.append(ele)

            
    return c_names

        

In [7]:
# FUNCTION: CREDIT REQUIREMENTS - finished

def find_credits(entry):
    # List of words not capitalised in headings
    conjunctions = ["and", "or", "but", "nor", "yet", "so", "for", "&", "and/or"]
    articles = ['a', 'an', 'the']
    prepositions = ["in", "to", "of", "at", "by", "up", "for", "off", "on"]
    no_cap = conjunctions + articles + prepositions
    years = ['year', 'first-year', 'second-year', 'third-year'] 
    years_cond = ['first', 'second', 'third', 'propedeutic', '1st', '2nd', '3rd', 'academic', 'Academic'] #needs to be separate because of problem: Clinical Psychology and the first
    
    cred_l = ['ec', 'ects', 'ect', 'credits', 'ect', 'ecs']
    credits = {}
    reverse = False
    for sentence in entry:       
        if 'ec' or 'ects' or 'ect' or 'credits' or 'ecs' in sentence: #check if the entry contains ec information

            words = sentence.split()
            for i in range(len(words)):
                
                if words[i] in cred_l: #find position of 'ec'/...
                    n_ecs = words[i-1] #Find the number of credits (element before 'ec')
                    
                    if n_ecs[0] == '(':
                        n_ecs = n_ecs[1:]
                        words = words[::-1]
                        i = len(words)-i
                        reverse = True
                    title = ''
                    co = 1
                    x = 1
                    
                    #Find course or year the credits belong to by going trough the words after ec
                    while x == 1 and (i+co) < len(words):
                        next_w = words[i+co] #next word


                    # Credits required from first/second/third year
                    
                        first_l = ['first-year', 'first', '1st', 'propedeutic'] #XXXX might find 'first'/.. that does not belong to year
                        second_l = ['second-year', 'second', '2nd']
                        third_l = ['third-year', 'third', '3rd']
                        
                        if next_w in first_l:
                            credits['First year'] = n_ecs # XXX credits[f'First year {'the_study_programme'}']
                            x = 0
                        elif next_w in second_l:
                            credits['Second year'] = n_ecs 
                            x = 0
                        elif next_w in third_l:
                            credits['Third year'] = n_ecs 
                            x = 0
                            
                            
                    # Credits required from a course(/specialisation): find course(/specialisation) name
                    
                        #check if course name starts by checking capitalisation
                        elif next_w[0] in upper:
                            co2 = 1
                            
                            if (i+co+co2) < len(words): #check if uppercase words is the last word, old error when last word capitalisation e.g., ['48 ec uit een propedeuse Geesteswetenschappen']
                           
                                #Find course names consisting of multiple words and conjunctions, articles and prepositions
                                while words[i+co+co2][0] in upper or words[i+co+co2] in no_cap or words[i+co+co2] in years or words[i+co+co2] in years_cond:
                                    next_w += " " + words[i+co+co2]
                                    if (i+co+co2+1) == len(words):
                                        break
                                    co2 += 1
                            
                            #Delete conjunctions/articles/prepositions in the end as these are likely not part of 
                            #the course name anymore: e.g 'Clinical Psychology and the'
                            title = next_w.split()
                            if reverse == True: # if reversed because course comes before credits e.g., Research Methods (60), bring back to normal order
                                title = title[::-1]
                            while len(title) != 0 and title[-1] in no_cap or title[-1] in years_cond: 
                                del title[-1]
                            
                            #Add title and credits to dictionary
                            title = ' '.join(title)
                            credits[title] = n_ecs 
                            x = 0

                        #one word course name
                        else:
                            co+=1
                            
                            #if course name not found
                            if (i+co) == len(words) and not title:
                                
                                # if the word after ecs is "in" or "of", extract the rest of the sentence as the credit requirement, e.g., "at least 30 ec in linguistics and/or language acquisition courses"
                                if words[i+1] == "in" or "of" or "for":
                                    sentence_rest = words[i+2:len(words)]
                                    title = ' '.join(sentence_rest)
                                    credits[title] = n_ecs                                 
                                else:
                                    title = f"There is a {n_ecs} credit requirement I could not specify"
                                    credits[title] = n_ecs 
                    
                    words = sentence.split()

    return credits   


In [8]:
url = f'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/100604'
html = simple_get(url)
dom = BeautifulSoup(html, 'html.parser')


In [9]:
# FINAL SCRAPER: ENTRY REQUIREMENTS 
# of one course

upper = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

for info_block in dom.find_all('tr', {'class':''}):
    info_block = info_block.text
    
    # Entry requirements of the course: full text
    if 'Entry requirements' in info_block: #Find text box with entry requirements
        entry = info_block.replace("Entry requirements", "") #Remove title of the text box
        entry = ' '.join(entry.split()) #remove multiple whitespaces by splitting and joining
        #entry = entry.lower()
        
        #Split into sentences as these contain seperate criteria
        entry = entry.replace(". ", ".").split(".")
        entry = list(filter(None, entry))
        entry = clean_entry(entry)

        # CREDIT REQUIREMENTS 
        cred_l = ['ec', 'ects', 'ect', 'credits', 'ect', 'ecs']
        credits = {}
        credits_dict = find_credits(entry)
        

        # COURSE REQUIREMENTS (having completed specific courses)
        req_courses = []
        for sentence in entry:
            words = sentence.split()
            new_courses = find_c_names(words)
            req_courses += new_courses


This will take very long to run

In [ ]:
"""
LIST SCRAPER
"""

#delete and continue on Scraper 2
#FINAL SCRAPER: name, urls, levels and fields (subset of the data) 

potential_levels = ["Bachelor's", "Dual Master's", "Master's", "Exchange Programme Exchange Programme UvA", 
                    "Exchange programme Exchange Programme", "Exchange programme Exchange Programme UvA", 
                    "Exchange Programme Exchange Programme", "pre-Master's programme", "Secondary Subject", 
                    "Bijvak", "Honours MSc", "Research Master's", "Minor"] #tailored to english website
upper = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"


courses_df = pd.DataFrame({'name': [], 'acad_level': [], 'study_field': [], 'required_courses': [], 'required_credits': [], 'url': []})
names, acad_levels, study_fields, required_courses, required_credits, urls = [],[],[],[],[],[]

unique_n = 94300 
    
while unique_n < 104000:
    print(f'course number: {unique_n}')
    url = f'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/{unique_n}'
    html = simple_get(url)
    dom = BeautifulSoup(html, 'html.parser')

    unique_n += 1

    if (dom.find('article', class_ = 'twelve columns')) is None: #no course under this url - go to next url
        pass

    else: #course exists
        urls.append(url)
        
        name = dom.find('article', class_ = 'twelve columns').h1.text[1::]
        names.append(name)
        
        fields = []
        levels_l = []
        req_cour = []
        req_credits = []
        
        for info_block in dom.find_all('tr', {'class':''}):
            info_block = info_block.text


            #Education level and field of the course
            if 'Is part of' in info_block: #Find text box that contains level and field of the course
                is_part_of = info_block.replace("Is part of", "") #Remove title of the text box

                # Education level of the course e.g., Bachelor's
                levels = ""
                #levels_l = []
                for l in potential_levels:
                    if l in is_part_of:
                        levels_l.append(l)
                        levels += l
                        #print(levels)
                        levels+=";"
                        
                levels = levels[:-1]

                
                # Field(s) of the course e.g., Business
                for lev in levels_l: 
                    is_part_of = is_part_of.replace(lev+" ", ";") #replace education level by semicolon to split on the semicolon later
                fields = is_part_of[1::].split(";") #remove first semicolon to not get an empty element in the study fields list 
                                                    #and split on semicolon to get this list                

                for i in range(len(levels_l)):
                    if 'Exchange' in levels_l[i]: #unified way of describing and exchange programme
                        levels_l[i] = 'Exchange programme'        
            
            
            # Entry requirements of the course: full text
            if 'Entry requirements' in info_block: #Find text box with entry requirements
                entry = info_block.replace("Entry requirements", "") #Remove title of the text box
                entry = ' '.join(entry.split()) #remove multiple whitespaces by splitting and joining

                #Split into sentences as these contain seperate criteria
                entry = entry.replace(". ", ".").split(".")
                entry = list(filter(None, entry))

                entry = clean_entry(entry)
                print(entry)

                # CREDIT REQUIREMENTS 
                cred_l = ['ec', 'ects', 'ect', 'credits', 'ect', 'ecs']
                credits = {}
                credits_dict = find_credits(entry)


                # COURSE REQUIREMENTS (having completed specific courses); XXX could be moved into function
                req_cour = []
                for sentence in entry:
                    words = sentence.split()
                    new_courses = find_c_names(words)
                    req_cour += new_courses

                req_credits.append(credits_dict)
                
        print(name, fields, levels_l, req_cour, req_credits, url)
                
        study_fields.append(fields)
        acad_levels.append(levels_l)       
        required_courses.append(req_cour)
        required_credits.append(req_credits)

        

course number: 94300
course number: 94301
course number: 94302
course number: 94303
course number: 94304
course number: 94305
['Being admitted to the BA Media en cultuur or the BA Media and Information']
Media Theory ['New Media & Digital Culture (Media and Information)', 'Media and Culture', 'Cultural Information Studies (Media and Information)pre-', 'programme Media Studies - English taught MA programmes'] ["Bachelor's", "Master's", "pre-Master's programme"] ['BA Media', 'BA Media and Information'] [{}] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94305
course number: 94306
course number: 94307
['At least 48 ec of a propedeutic year obtained', 'You will be automatically allowed access to the course if you have demonstrable knowledge of linguistics (e', 'g', 'you completed the course Inleiding Taalwetenschap)', 'Otherwise, you can contact the coordinators on how to obtain sufficient background in linguistics (see below for details)']
Neurolinguistics ['Col

course number: 94377
course number: 94378
course number: 94379
['A positive binding study advice (preferably in Humanities)', "For exchange students: An equivalent level of the 'propedeuse' (i", 'e', '60 ec in courses on first year level)', 'Priority for exchange students']
The Five Senses of Cinema ['College of Humanities (BA)'] ['Exchange programme'] ['Humanities)', 'first year', 'Priority'] [{'First year': '60'}] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94379
course number: 94380
course number: 94381
course number: 94382
course number: 94383
course number: 94384
course number: 94385
course number: 94386
course number: 94387
course number: 94388
course number: 94389
course number: 94390
course number: 94391
Institutional and Substantive Law of the EU ['European Studies', 'College of Humanities (BA)', 'European Law'] ["Bachelor's", 'Exchange programme', 'Minor'] [] [] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94391
cours

course number: 94473
['A positive binding study advice (BSA) or dispensation from the BSA']
The Sublime ['College of Humanities (BA)'] ['Exchange programme'] ['BSA'] [{}] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94473
course number: 94474
course number: 94475
course number: 94476
course number: 94477
course number: 94478
course number: 94479
course number: 94480
course number: 94481
course number: 94482
course number: 94483
['Admission to Bachelor’s in European Studies (in English)']
The Other Europe ['European Studies', 'College of Humanities (BA)'] ["Bachelor's", 'Exchange programme'] ['Admission to Bachelor’s in European Studies', 'English)'] [{}] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94483
course number: 94484
course number: 94485
course number: 94486
course number: 94487
course number: 94488
course number: 94489
course number: 94490
course number: 94491
Narratological and Discursive Analysis  ['College of Humanit

course number: 94543
['Sequence requirement: Modern Hebrew Language Acquisition 1 or equivalent (please contact the lecturer)']
Modern Hebrew Language Acquisition 2 ['European Studies', 'College of Humanities (BA)', 'Linguistics'] ["Bachelor's", 'Exchange programme'] ['Sequence', 'Modern Hebrew Language Acquisition'] [{}] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94543
course number: 94544
course number: 94545
['Sequence requirement: Modern Hebrew Language Acquisition 2 or equivalent (please contact the lecturer)']
Modern Hebrew Language Acquisition 3 ['European Studies', 'College of Humanities (BA)', 'Linguistics'] ["Bachelor's", 'Exchange programme'] ['Sequence', 'Modern Hebrew Language Acquisition'] [{}] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94545
course number: 94546
course number: 94547
['Sequence requirement: Modern Hebrew Language Acquisition 3 or equivalent (please contact the lecturer)']
Modern Hebrew Language

course number: 94601
course number: 94602
course number: 94603
course number: 94604
course number: 94605
course number: 94606
course number: 94607
course number: 94608
course number: 94609
course number: 94610
course number: 94611
course number: 94612
course number: 94613
course number: 94614
course number: 94615
course number: 94616
course number: 94617
course number: 94618
course number: 94619
['A positive binding study advice (BSA) or dispensation from the BSA']
Privacy: Theoretical Perspectives, Future Challenges ['College of Humanities (BA)', 'Privacy Studies'] ['Exchange programme', 'Minor'] ['BSA'] [{}] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94619
course number: 94620
course number: 94621
['At least 48 ec of first year European Studies completed']
Philosophy of the Humanities (Area Studies) ['European Studiespre-', 'programme Literary Studies'] ["Bachelor's", "Master's", "pre-Master's programme"] ['first year European Studies'] [{'First year': 

course number: 94655
Language Proficiency Sign Language of the Netherlands 5 ['Sign Language Linguistics (Linguistics)'] ["Bachelor's"] [] [] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94655
course number: 94656
course number: 94657
['Introduction to Linguistics (or equivalent)']
Phonology and Morphology of Sign Languages ['College of Humanities (BA)', 'Sign Language Linguistics (Linguistics)'] ["Bachelor's", 'Exchange programme'] ['Introduction to Linguistics'] [{}] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94657
course number: 94658
course number: 94659
Language Proficiency Sign Language of the Netherlands 6 ['College of Humanities (BA)', 'Sign Language Linguistics (Linguistics)'] ["Bachelor's", 'Exchange programme'] [] [] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94659
course number: 94660
course number: 94661
['Introduction to Linguistics; Introduction to Sign Linguistics']
Lexicon and Sy

course number: 94754
Vertalen Noors - Nederlands [] [] [] [] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94754
course number: 94755
course number: 94756
Vertalen Zweeds - Nederlands [] [] [] [] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94756
course number: 94757
course number: 94758
course number: 94759
course number: 94760
course number: 94761
['A positive binding study advice for the bachelor Media and Information']
Histories of Digital Culture ['New Media & Digital Culture (Media and Information)', 'College of Humanities (BA)pre-', 'programme Media Studies - English taught MA programmes'] ["Bachelor's", "Master's", 'Exchange programme', "pre-Master's programme"] ['Media and Information'] [{}] https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94761
course number: 94762
course number: 94763
English Linguistics 4: Research in English Linguistics ['English Language and Culture', 'College of Humanities 

#### Input your own courses

In [ ]:
courses_df = pd.DataFrame({'name': names, 'acad_level': acad_levels, 'study_field': study_fields, 'req_courses': required_courses, 'req_credits': required_credits, 'url': urls})

display(courses_df)


In [ ]:
my_courses = {'Research Methods and Statistics': 12, 'Introductory Psychology and Cognition': 12, 'Developmental Psychology': 6, 
              'Social and Work & Organisational Psychology': 12, 'Clinical Psychology & Brain and Cognition': 12, 
              'Professional Orientation, Research Participation, and Propaedeutic Thesis': 6, 'History of the modern world since 1750': 6,
              'Honours Programme: Breaking Borders 1': 3, 'Scientific and Statistical Reasoning': 9, 'Introduction to political science': 6, 
              'Political Psychology': 6, 'Public Policy and Governance': 6, 'Practical training: Psychological Communication': 3, 
              'Academic skills for political scientists I': 6, 'Fundamentals of Psychology': 6, 'Research project I': 6,
              'Practical training: Psychological Research': 12, 'Emotion': 6, 'Motivation': 6, 'History of political thought': 6,
              'Groups and Intergroup Behaviour': 6, 'Attitudes and Behavioural Change': 6, 'Honoursmodule: China, The Rising Power': 6,
              'Introduction to political science research': 9, 'International relations': 6, 'Academic skills for political scientists II': 3,
              'Applied SP: Towards a Sustainable Society': 6, 'Law and politics': 6, 'Honours programme: Internship': 3,
              'Political economy': 6, 'Methodology: Operationalisation, Design and Analyses': 6, 'Current Topics: Stress in Health and Disease': 6,
              'Honours Programme: Breaking Borders 2': 3, 'Research methods': 9, 'Democracies, autocracies and transitions': 6, 
              'Current Topics: Psychology and AI': 6, 'Academic skills for political scientists III': 9, "Bachelor's Thesis Social Psychology": 12,
              'Diversity, equality and justice': 6, 'Latent Variable Models': 6, 'Fundamentals of Methodology': 6,
              'Violence & security. Paradigms and debates': 6, 'Multivariate Statistics & Machine Learning': 6,
              'Honoursmodule: Urban transformation, gentrification and inequality in Amsterdam': 6, 'Research project II: Practicing research skills': 6,
              'Bayesian Statistics': 6, 'Basic Skills in Mathematics, Programming & Statistics': 6, 'Scientific Programming 1': 3,
              'Scientific Programming 2': 3, 'Politics and Artificial Intelligence': 6}

In [ ]:
def filter_tkinter(level, study_field):
    filters = {'level': level, 'study field': study_field, 'year': 'first-year'}

    filtered = pd.DataFrame()

    for index, row in courses_df.iterrows():
        for ele in row['acad_level']:
            if ('level', ele) in filters.items(): #filtered right academic level
                for ele in row['study_field']:
                    if ('study field', ele) in filters.items(): #filtered right study field
                        filtered = filtered.append(row, ignore_index=True)

    return filtered

#Find a %like% function, not necessary if drop down    

In [ ]:
 #currently adds all courses where you have completed one of the required courses

def filter_elegibility(filtered):
    elegible = pd.DataFrame()

    for index, row in filtered.iterrows():
        if len(row['req_courses'])==0:
            elegible = elegible.append(row, ignore_index=True)
            print("yes, because no requirements")
        else:
            for ele in row['req_courses']:
                if ele in my_courses: #has to be exact? XXX
                    elegible = elegible.append(row, ignore_index=True)
                    print(f"yes, {row['name']}")
    return elegible



In [ ]:
display(filtered)


## GUI 



### tkinter

In [ ]:
#prepare dropdown for tkinter
fields_unique = []
levels_unique = []

for index, row in courses_df.iterrows():
    for ele in row['study_field']:
        #print (ele)
        if ele not in fields_unique:
            fields_unique.append(ele)
    for ele in row['acad_level']:
        #print (ele)
        if ele not in levels_unique:
            levels_unique.append(ele)
        
fields_unique = sorted(fields_unique)
levels_unique = sorted(levels_unique)
print(fields_unique)         
print(levels_unique)

In [ ]:
import tkinter as tk
from tkinter import ttk


In [ ]:

def on_l_select(option):
    global selected_level
    selected_level = option

def on_f_select(option):
    global selected_field
    selected_field = option


    
def submit(): #submit the level and field selection 
    if selected_level in levels_unique and selected_field in fields_unique:
        print("Selected level:", selected_level)
        print("Selected field:", selected_field)
        
        global filtered
        filtered = filter_tkinter(selected_level, selected_field) #filtered df
        global eligibile
        eligibile = filter_elegibility(filtered)
        #eligibile=filtered USE FOR TESTING
        names = eligibile['name'].tolist() #filtered course names
        root.destroy()
        your_courses(names, eligibile)

def your_courses(names, eligibile): #display filtered course names    
    global root2
    root2 = tk.Tk()
    root2.title("Your options!")

    for name in names:
        button2 = tk.Button(root2, text=name, command=lambda name=name: submit2(name), font=font_style)
        button2.pack(pady=0)
    root2.mainloop()

        
    #display(eligibile)

def open_url(event):
    webbrowser.open(row['url'].item())
    
def submit2(course_name):
    #Show more information based on the selected course name
    course_info = "Course info"
    
    #New window
    info_window = tk.Toplevel(root2)
    info_window.title(course_name)
    
    #Display the course information
    label = tk.Label(info_window, text=course_name + "\n", font=("Helvetica", 12, "bold"), justify="center")
    label.pack()
    
    # Display additional information
    display(eligibile)
    row = eligibile.loc[eligibile['name'] == course_name]
    display(row)
    
    if len(row['req_courses'].item()) == 0: #Display if no courses required
        courses_label = tk.Label(info_window, text="Required courses:\nNo courses required\n")
        courses_label.pack()
    else:
        courses_label = tk.Label(info_window, text="Required courses:\n" + '\n'.join(row['req_courses'].item()))
        courses_label.pack()
    
    if len(row['req_credits'].item()) == 0: #Display if no courses required
        credits_label = tk.Label(info_window, text="\nRequired credits:\nNone of the courses requires a specific number of credits\n")
        credits_label.pack()
    else:
        requirements = []
        req_credits_dict = row['req_credits'].item()[0]  #Access the dictionary inside the list
        for key, value in req_credits_dict.items():
            requirement = key + ': ' + value + '\n'
            requirements.append(requirement)

        credits_label = tk.Label(info_window, text="\nRequired credits:\n" + ''.join(requirements))
        credits_label.pack()
    
    #for key, value in row['url']:
    url_label = tk.Label(info_window, text=row['url'].item(), fg='blue', cursor='hand2')
    url_label.pack()
    url_label.bind("<Button-1>", open_url)
    
    info_window.mainloop()

        
        
root = tk.Tk()

# Style
root.geometry("400x300")  # Window size
root.title("Course Selection")  # Window title
root.configure(bg="white")  # Window background color

# Font and color for dropdown menus
font_style = ("Helvetica", 12)

# Dropdown for levels
options = levels_unique
selected_level = tk.StringVar(root)
selected_level.set("Level")  # Set initial selected option
dropdown_level = tk.OptionMenu(root, selected_level, *options, command=on_l_select)
dropdown_level.pack(pady = 2)  # Increase distance from the dropdowns
dropdown_level.config(height=2, width=30, font = ("Helvetica", 12)) 

# Dropdown for fields
options = fields_unique
selected_field = tk.StringVar(root)
selected_field.set('Field of studies')  # Set initial selected option
dropdown_field = tk.OptionMenu(root, selected_field, *options, command=on_f_select)
dropdown_field.pack(pady = 2)  # Increase distance from the dropdowns
dropdown_field.config(height=2, width=30, font = ("Helvetica", 12)) 

# Submit button
submit_button = tk.Button(root, text="Find courses", command=submit, font=font_style)
submit_button.pack(pady=20)  # Increase distance from the dropdowns
submit_button.config(height=1, width=20, font = ("Helvetica", 12, 'bold'))  # Make the button bigger

root.mainloop()




In [ ]:
#df = pd.DataFrame(courses)
df.to_csv('course_data.csv', index=False)